In [1]:
!pip install z3-solver

  Using cached z3_solver-4.12.2.0-py2.py3-none-win_amd64.whl (57.9 MB)


ERROR: Could not install packages due to an OSError: [WinError 5] Accesso negato: 'C:\\Python39\\bin'
Consider using the `--user` option or check the permissions.



In [2]:
from z3 import *
import numpy as np
from itertools import combinations

In [3]:
def at_least_one(bool_vars):
    return Or(bool_vars)

def at_most_one(bool_vars):
    return [Not(And(pair[0], pair[1])) for pair in combinations(bool_vars, 2)]

def exactly_one(bool_vars):
    return at_most_one(bool_vars) + [at_least_one(bool_vars)]

def at_most_k_seq(x, k, name):
    clauses = []
    n = len(x)
    s =[[Bool(f"s_{i}_{j}_{name}") for j in range(k)] for i in range(n - 1)]
    clauses.append(Or(Not(x[0]), s[0][0]))
    clauses += [Not(s[0][j]) for j in range(1, k)]
    clauses.append(Or( Not(x[n - 1]), Not(s[ n-2 ][ k-1 ])))
    for i in range(1, n - 1):
      clauses.append(Or(Not(x[i]), s[i][0]))
      clauses.append(Or(Not(s[i-1][0]), s[i][0]))
      for j in range(1, k):
        clauses.append(Or(Not(s[i-1][j]), s[i][j]))
        clauses.append(Or(Not(x[i]), Not(s[i-1][j-1]), s[i][j]))
      clauses.append(Or(Not(x[i]), Not(s[ i-1 ][ k-1 ])))
    return And(clauses) 

In [4]:
def multiple_couriers(m, n, D, l, s, min_dist): #m is the number of couriers, n is the number of packages, D is the distance matrix, l is the load of each courier, s is the size of each package

  # i is the courier
  # j is the step
  # k is the one-hot encoding of the package
  c = [[[Bool(f"c_{i}_{j}_{k}") for k in range(n+1)] for j in range(n+2)] for i in range(m)]

  # i is the courier
  # j is the package
  # k is the bitwise encoding of the package
  w = [[[Bool(f"w_{i}_{j}_{k}") for k in range(s[j])] for j in range(n)] for i in range(m)]

  # i is the courier
  # start is a position of the route
  # end is the next position of the route
  d = [[[Bool(f"d_{i}_{start}_{end}") for end in range(n+1)] for start in range(n+1)] for i in range(m)]

  # Defining the solver instance
  solver = Solver()

  # Binding the two matrixes
  for i in range(m):
    for j in range(1, n+1):
      for package in range(n):
        for k in range(s[package]):
          solver.add(Implies(c[i][j][package], w[i][package][k]))
          # solver.add(Implies(w[i][package][k], c[i][j][package]))

  # Binding the routing
  for i in range(m):
    for j in range(n):
      for start in range(n+1):
        for end in range(n+1):
          solver.add(Implies(And(c[i][j][start], c[i][j+1][end]), d[i][start][end]))

  # At each time, the courier carries exactly one package or he is at base
  for i in range(m):
    for j in range(n+2):
      solver.add(exactly_one(c[i][j][:]))

  # Each package is carried only once
  for package in range(n):
    vars = []
    for i in range(m):
      for j in range(1, n+1):
        vars.append(c[i][j][package])
    solver.add(exactly_one(vars))

  # The total weight carried by each courier must be less or equal than his
  # carriable weight
  for i in range(m):
    vars = []
    for j in range(n):
      for k in range(s[j]):
        vars.append(w[i][j][k])     
    solver.add(at_most_k_seq(vars, l[i], f"weight_{i}"))

  # At start/end the courier must be at the base
  for i in range(m):
    solver.add(c[i][0][n])
    solver.add(c[i][n+1][n])

  # Getting the maximum distance
  for i in range(m):
    dists = []
    for start in range(n+1):
      for end in range(n+1):
        for z in range(D[start][end]):
          dists.append(d[i][start][end])
    solver.add(at_most_k_seq(dists, min_dist, f"Courier_dist_{i}"))

  ## Breaking Symmetry ##

  # Lexicographic order for each courier
  for i in range(m):
    for j in range(1, n):
      for z in range(j+1, n+1):
        solver.add(Implies(c[i][j][n], c[i][z][n]))
      
  sol = solver.check()
  if sol == sat: #If it's sat, we want to print the assignment
    g = solver.model()
    print(solver.check())
    # for i in range(m):
    #   print()
    #   print(f"Courier {i+1}:")
    #   for j in range(n+2):
    #     for k in range(n+1):
    #       if g[c[i][j][k]]:
    #         print(f"Time {j} Place {k+1} : {g[c[i][j][k]]}")
    return g
  else:
    print("Not sat")
    return False

In [6]:
from time import time

MAXITER = 500

def minimizer_binary(instance, solver=multiple_couriers, maxiter = MAXITER):
  m = instance["m"]
  n = instance["n"]
  D = instance["D"]
  l = instance["l"]
  s = instance["s"]
  start = time()

  min_dist = 9999999
  for i in range(len(D)):
    for j in range(len(D[i])):
      if D[i][j] <= min_dist and D[i][j] != 0:
        min_dist = D[i][j]
    
  max_dist = 0
  for i in range(len(D)):
    max_dist += max(D[i])
  
  iter = 1
  # TODO push e op del constraint con k invece di creare un nuovo modello
  while iter < maxiter:
    k = int((min_dist + max_dist) / 2)
    sol = solver(m, n, D, l, s, k)
    if not sol:
      min_dist = k
    else:
      max_dist = k
    if abs(min_dist - max_dist) <= 1:
      if sol:
        return k, sol, f"{time() - start:.2f}", iter
      else:
        return 0, "Unsat", f"{time() - start:.2f}", iter
    iter+=1
  return 0, "Out of time", f"{time() - start:.2f}", iter
  

In [7]:
instance = {
  "m" : 2,
  "n" : 6,
  "D" : [[0, 3, 4, 5, 6, 6, 2],
      [3, 0, 1, 4, 5, 7, 3],
      [4, 1, 0, 5, 6, 6, 4],
      [4, 4, 5, 0, 3, 3, 2],
      [6, 7, 8, 3, 0, 2, 4],
      [6, 7, 8, 3, 2, 0, 4],
      [2, 3, 4, 3, 4, 4, 0]],
  "l" : [15, 10],
  "s" : [3, 2, 6, 5, 4, 4]
}

In [8]:
min_dist, sol, time_passed, iter = minimizer_binary(instance)
print("\nResults:")
if type(sol) == str: 
  print(f"Solution: {sol}")
print(f"Minimum distance found is {min_dist}")
print(f"Time: {time_passed}")
print(f"Iterations: {iter}")

sat
Not sat
sat
Not sat
sat

Results:
Minimum distance found is 14
Time: 26.88
Iterations: 5
